# Import Libraries

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
import os

In [6]:
print(os.getcwd())

c:\Users\Javi\Desktop\CSE 120\Anomaly Detection\Leaf-Classification\data\tomato


In [7]:
train_data_dir = 'train'

# Check if the directory exists
if not os.path.exists(train_data_dir):
    print(f"The directory {train_data_dir} does not exist.")
else:
    print(f"Found directory {train_data_dir}. Proceeding with loading data...")
    # Proceed with your data loading and model training


Found directory train. Proceeding with loading data...


# Data loading and preprocessing

In [8]:
train_data_dir = 'train'
test_data_dir = 'val'

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3
)

In [10]:
train_gen = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(112, 112),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 7000 images belonging to 10 classes.


In [11]:
val_gen = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(112, 112),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 3000 images belonging to 10 classes.


In [12]:
test_datagen = ImageDataGenerator(rescale=1/255.0)

In [13]:
test_gen = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(112, 112),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 1000 images belonging to 10 classes.


# Model architecture

In [14]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(112, 112, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # Assuming we have 10 classes, adjust if necessary
])

# Compile the model

In [15]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Early Stopping to avoid overfitting

In [16]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    verbose=1,
    restore_best_weights=True
)

# Train the model

In [17]:
history = model.fit(
    train_gen,
    epochs=80,
    validation_data=val_gen,
    callbacks=[early_stopping]
)

Epoch 1/80
219/219 [==============================] - 42s 190ms/step - loss: 1.9853 - accuracy: 0.2716 - val_loss: 1.7307 - val_accuracy: 0.3637
Epoch 2/80
219/219 [==============================] - 37s 171ms/step - loss: 1.4925 - accuracy: 0.4641 - val_loss: 1.3524 - val_accuracy: 0.5193
Epoch 3/80
219/219 [==============================] - 38s 171ms/step - loss: 1.2941 - accuracy: 0.5364 - val_loss: 1.2630 - val_accuracy: 0.5307
Epoch 4/80
219/219 [==============================] - 38s 172ms/step - loss: 1.1679 - accuracy: 0.5766 - val_loss: 1.0988 - val_accuracy: 0.6113
Epoch 5/80
219/219 [==============================] - 36s 166ms/step - loss: 1.0310 - accuracy: 0.6234 - val_loss: 0.9101 - val_accuracy: 0.6737
Epoch 6/80
219/219 [==============================] - 37s 170ms/step - loss: 0.9292 - accuracy: 0.6611 - val_loss: 0.8507 - val_accuracy: 0.6910
Epoch 7/80
219/219 [==============================] - 35s 158ms/step - loss: 0.8413 - accuracy: 0.6916 - val_loss: 0.7925 - val_ac


# Evaluate the model

In [18]:
scores = model.evaluate(test_gen)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

32/32 [==============================] - 1s 41ms/step - loss: 0.4530 - accuracy: 0.8990
Test loss: 0.4529883861541748
Test accuracy: 0.8989999890327454


# Save the model

In [19]:
model.save('merged_model.h5')